#### Import required libraries

In [2]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure you have downloaded the necessary NLTK data files
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jvazq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Upload dataset from csv

In [3]:
# Load the file youtoxic_english_1000.csv to youtoxic dataframe
youtoxic = pd.read_csv('youtoxic_english_1000.csv')


#### Review dataset

In [4]:
youtoxic.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
youtoxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(3)
memory usage: 35.3+ KB


#### Convertir Booleanos a Int64 por si alguna librería posterior no puede hacer la transformación intrínseca de boolean a int64

In [6]:
# Identify boolean columns
bool_columns = youtoxic.select_dtypes(include=['bool']).columns

# Convert boolean columns to int64
youtoxic[bool_columns] = youtoxic[bool_columns].astype('int64')

# Display the updated information about the dataset to verify the conversion
youtoxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   int64 
 4   IsAbusive        1000 non-null   int64 
 5   IsThreat         1000 non-null   int64 
 6   IsProvocative    1000 non-null   int64 
 7   IsObscene        1000 non-null   int64 
 8   IsHatespeech     1000 non-null   int64 
 9   IsRacist         1000 non-null   int64 
 10  IsNationalist    1000 non-null   int64 
 11  IsSexist         1000 non-null   int64 
 12  IsHomophobic     1000 non-null   int64 
 13  IsReligiousHate  1000 non-null   int64 
 14  IsRadicalism     1000 non-null   int64 
dtypes: int64(12), object(3)
memory usage: 117.3+ KB


In [7]:
# Export the youtoxic dataframe to an Excel file
youtoxic.to_excel('youtoxic.xlsx', index=False)

##### Después de analizar el fichero en Excel se tienen estas conclusiones:

* Efectivamente IsToxic es la bandera que agrupa a los diferentes tipos de clasificaciones de odio.
* No hay ningún discurso de odio que no tenga la bandera IsToxic encendida.
* No hay ninguna bandera IsToxic encendida sin que haya ninguna de las otras banderas encendidas. Esto significa que la clasificación IsToxic no existe per se y solo representa la existencia de discurso de odio en alguna de las características.

#### Porcentaje de registros en cada categoria

In [8]:
# Get total number of rows
total_rows = len(youtoxic)

# Select integer columns
int_cols = youtoxic.select_dtypes(include=['int64']).columns

# Calculate counts and percentages
results = []
for col in int_cols:
    count_ones = youtoxic[col].sum()  # Sum of 1s
    percentage = (count_ones / total_rows) * 100
    results.append({
        'Column': col,
        'Count of 1s': count_ones,
        'Percentage': f'{percentage:.2f}%'
    })

# Create and display table
table = pd.DataFrame(results)
display(table)

,Column,Count of 1s,Percentage
0,IsToxic,462,46.20%
1,IsAbusive,353,35.30%
2,IsThreat,21,2.10%
3,IsProvocative,161,16.10%
4,IsObscene,100,10.00%
5,IsHatespeech,138,13.80%
6,IsRacist,125,12.50%
7,IsNationalist,8,0.80%
8,IsSexist,1,0.10%
9,IsHomophobic,0,0.00%


##### Note: IsToxic es la columna bandera que indica si hay algún comentario de odio en las otras 11 categorias. No se necesita crear una columna que sumarice 12 columnas porque esa columna ya está en el dataset.

* Los datos no nos permiten clasificar un discurso por medio de las características: - 
- IsHomophobic
- IsRadicalism

* Las características 
- IsSexist
- IsNationalist
- IsReligiousHate
- IsThreat 
están severamente desbalanceadas.

* También están muy desbalanceadas las características: 
- IsProvocative
- IsObscene
- IsHateSpeech
- IsRacist

* La única característica donde el desbalanceo es razonable es:
- IsAbusive

* Con estas observaciones se presume que el modelo con una sola bandera IsToxic es muy generalista y que por lo tanto no es capaz de predecir con una mejor precisión (alrededor de 70%) si el mensaje es de odio o no.

* Un modelo multi-label binary classification (entiendo que con un Naive Bayes) puede ser una mejor solución en este caso, utilizando solamente las características:
- IsAbusive
- IsProvocative
- IsObscene
- IsHateSpeech
- IsRacist

Por supuesto, resolviendo el problema del desbalanceo en las últimas 4 características.

En cualquier caso sí es importante utilizar la característica VideoId como información útil para el modelo.

---
A partir de aquí se preparan 2 datasets. Uno para entrenar un modelo multi-etiqueta de clasificación binaria. El otro para un modelo de una sola categoria: Istoxic

Youmultihatred:
- VideoId
- Text
- IsAbusive
- IsProvocative
- IsObscene
- IsHateSpeech
- IsRacist

Youtoxic:
- VideoId
- Text
- IsToxic

In [9]:
# List of columns to keep for version without multi-label classification

youmultihatred = youtoxic[['VideoId', 'Text', 'IsAbusive','IsProvocative','IsObscene','IsHatespeech', 'IsRacist']]

youtoxic = youtoxic[['VideoId', 'Text', 'IsToxic']]


#### Preprocess dataset

1. Remover URLs
2. Remover special characters y números
3. Convertir a minúsculas
4. Remover espacios innecesarios
Tokenizar momentaneamente para:
5. Quitar Stopwords
6. Lematizar

In [10]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# No utilizaremos textblob dado que las metricas mejoraron al no utilizarlo.
# Ademas, textblob es muy lento para procesar grandes cantidades de texto.

#from textblob import TextBlob

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Correct misspellings
    #text = str(TextBlob(text).correct())
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the words back into a single string
    text = ' '.join(words)

    return text


# Apply the preprocess_text function to the 'Text' column of the youtoxic and youmultihatred dataframes
youtoxic['Text'] = youtoxic['Text'].apply(preprocess_text)
youmultihatred['Text'] = youmultihatred['Text'].apply(preprocess_text)

# Display the updated dataframes
youtoxic.head()
youmultihatred.head()

,VideoId,Text,IsAbusive,IsProvocative,IsObscene,IsHatespeech,IsRacist
0,04kJtp6pVXI,people would take step back make case wasnt an...,0,0,0,0,0
1,04kJtp6pVXI,law enforcement trained shoot apprehend traine...,1,0,0,0,0
2,04kJtp6pVXI,dont reckon black life matter banner held whit...,1,0,1,0,0
3,04kJtp6pVXI,large number people like police officer called...,0,0,0,0,0
4,04kJtp6pVXI,arab dude absolutely right shot extra time sho...,0,0,0,0,0


#### Generate sample csv file from dataframe

In [11]:
# Export a random selection of 20 rows from youtoxic dataframe to a CSV file
# Uncomment the line below to export the sample

youtoxic.head(n=20).to_csv('youtoxic_sample.csv', index=False)
youmultihatred.head(n=20).to_csv('youmultihatred_sample.csv', index=False)

### Building the ML models

##### 1.1 Modelo de ML utilizando solo la característica Text. IsToxic es la etiqueta

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# Preprocess the text data
X = youtoxic['Text']
y = youtoxic['IsToxic']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_train_pred = model.predict(X_train_tfidf)
y_test_pred = model.predict(X_test_tfidf)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

# Calculate overfitting percentage
overfitting_percentage = ((train_accuracy - test_accuracy) / train_accuracy) * 100

print(f'Training Accuracy: {train_accuracy:.2f}')
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Overfitting Percentage: {overfitting_percentage:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')


# Optimize hyperparameters using cross-validation
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

print(f'Best hyperparameters: {grid_search.best_params_}')
print(f'Best cross-validation score: {grid_search.best_score_:.2f}')


Training Accuracy: 0.93
Test Accuracy: 0.69
Overfitting Percentage: 25.57%
Precision: 0.80
Recall: 0.57
F1-score: 0.67
Best hyperparameters: {'C': 10, 'penalty': 'l2'}
Best cross-validation score: 0.69


#### 1.2 Modelo de ML utilizando las características Text y VideoId. IsToxic es el target.

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Preprocess the text data
X_text = youtoxic['Text']
X_video_id = youtoxic['VideoId']
y = youtoxic['IsToxic']

# Encode the video IDs
label_encoder = LabelEncoder()
X_video_id_encoded = label_encoder.fit_transform(X_video_id)

# Split the data into train and test sets
X_text_train, X_text_test, X_video_id_train, X_video_id_test, y_train, y_test = train_test_split(X_text, X_video_id_encoded, y, test_size=0.2, random_state=42)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_text_train_tfidf = vectorizer.fit_transform(X_text_train)
X_text_test_tfidf = vectorizer.transform(X_text_test)

# Combine text and video ID features
X_train = np.hstack((X_text_train_tfidf.toarray(), X_video_id_train.reshape(-1, 1)))
X_test = np.hstack((X_text_test_tfidf.toarray(), X_video_id_test.reshape(-1, 1)))

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

# Calculate overfitting percentage
overfitting_percentage = ((train_accuracy - test_accuracy) / train_accuracy) * 100

print(f'Training Accuracy: {train_accuracy:.2f}')
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Overfitting Percentage: {overfitting_percentage:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')


# Optimize hyperparameters using cross-validation
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f'Best hyperparameters: {grid_search.best_params_}')
print(f'Best cross-validation score: {grid_search.best_score_:.2f}')

Training Accuracy: 0.93
Test Accuracy: 0.71
Overfitting Percentage: 23.24%
Precision: 0.81
Recall: 0.60
F1-score: 0.69
Best hyperparameters: {'C': 10, 'penalty': 'l2'}
Best cross-validation score: 0.69


#### 2.1 Modelo de Machine Learning de clasificación binaria de multi etiquetas 

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss
import re
import warnings
warnings.filterwarnings('ignore')


class HateSpeechClassifier:
    def __init__(self):
        # Initialize encoders and vectorizer
        self.video_id_encoder = LabelEncoder()
        self.tfidf = TfidfVectorizer(max_features=5000)
        self.thresholds = {
            'IsAbusive': 0.5,
            'IsProvocative': 0.5,
            'IsObscene': 0.5,
            'IsHatespeech': 0.5,
            'IsRacist': 0.5
        }
        

    def prepare_features(self, df, fit_tfidf=True):
        """Prepare features from input dataframe"""
        # Encode VideoId
        video_ids_encoded = self.video_id_encoder.fit_transform(df['VideoId'])
        
        # Create TF-IDF features
        if fit_tfidf:
            text_features = self.tfidf.fit_transform(df['Text'])
        else:
            text_features = self.tfidf.transform(df['Text'])
        
        # Combine features
        return np.hstack((
            video_ids_encoded.reshape(-1, 1),
            text_features.toarray()
        ))

    def prepare_features_predict(self, df):
        """Prepare features from input dataframe for prediction"""
        # Encode VideoId
        X = self.prepare_features_predict(df)
        
        # Create TF-IDF features
        text_features = self.tfidf.transform(df['Text'])
        
        # Combine features
        return np.hstack((
            video_ids_encoded.reshape(-1, 1),
            text_features.toarray()
        ))

    def train(self, df):
        """Train the multi-label classification model"""
        # Prepare features
        X = self.prepare_features(df)
        
        # Prepare target variables
        y = df[['IsAbusive', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist']].values
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Define base classifier and parameter grid for optimization
        base_classifier = RandomForestClassifier(random_state=42)
        param_grid = {
            'estimator__n_estimators': [100, 200],
            'estimator__max_depth': [10, 20],
            'estimator__min_samples_split': [2, 5],
            'estimator__min_samples_leaf': [1, 2]
        }

        # Create multi-output classifier
        self.classifier = MultiOutputClassifier(base_classifier)

        # Perform grid search with cross-validation
        grid_search = GridSearchCV(
            self.classifier,
            param_grid,
            cv=5,
            scoring='accuracy',
            n_jobs=-1
        )

        # Fit the model
        grid_search.fit(X_train, y_train)
        self.classifier = grid_search.best_estimator_

        # Calculate and display metrics
        self._evaluate_model(X_train, X_test, y_train, y_test)
        
        return self

    def _evaluate_model(self, X_train, X_test, y_train, y_test):
        """Evaluate model performance and display metrics"""
        # Training predictions
        y_train_pred = self.classifier.predict(X_train)
        train_accuracy = accuracy_score(y_train.flatten(), y_train_pred.flatten())

        # Test predictions
        y_test_pred = self.classifier.predict(X_test)
        test_accuracy = accuracy_score(y_test.flatten(), y_test_pred.flatten())

        # Calculate overfitting percentage
        overfitting_percentage = ((train_accuracy - test_accuracy) / train_accuracy) * 100

        # Perform cross-validation
        cv_scores = cross_val_score(self.classifier, X_train, y_train, cv=5)

        print("\nModel Performance Metrics:")
        print("-" * 50)
        print(f"Training Accuracy: {train_accuracy:.4f}")
        print(f"Test Accuracy: {test_accuracy:.4f}")
        print(f"Overfitting Percentage: {overfitting_percentage:.2f}%")
        print(f"Cross-validation Scores: {cv_scores}")
        print(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        print("\nClassification Report:")
        print(classification_report(y_test, y_test_pred, 
              target_names=['IsAbusive', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist']))
        print(f"Hamming Loss: {hamming_loss(y_test, y_test_pred):.4f}")

    def predict(self, df):
        """Make predictions on new data"""
        # Prepare features
        X = self.prepare_features(df)
        
        # Get raw predictions
        raw_predictions = self.classifier.predict_proba(X)
        
        # Apply thresholds and combine results
        final_predictions = []
        for sample_predictions in zip(*raw_predictions):
            label_predictions = {}
            is_hatred = False
            
            for idx, (label, pred_probs) in enumerate(zip(
                ['IsAbusive', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist'],
                sample_predictions
            )):
                # Get probability of positive class
                pos_prob = pred_probs[1]
                # Apply threshold
                is_positive = pos_prob >= self.thresholds[label]
                label_predictions[label] = is_positive
                
                # If any label is positive, mark as hatred
                if is_positive:
                    is_hatred = True
                    
            label_predictions['IsHatred'] = is_hatred
            final_predictions.append(label_predictions)
            
        return final_predictions

# Example usage
if __name__ == "__main__":
    # Load data
    df = youmultihatred

    # Initialize and train classifier
    classifier = HateSpeechClassifier()
    classifier.train(df)
    
    # Make predictions on sample data
    sample_predictions = classifier.predict(df)
    
    '''
    print("\nSample Predictions:")
    for i, pred in enumerate(sample_predictions):
        print(f"\nText {i+1} Predictions:")
        for label, value in pred.items():
            print(f"{label}: {value}")
    '''


Model Performance Metrics:
--------------------------------------------------
Training Accuracy: 0.8825
Test Accuracy: 0.8070
Overfitting Percentage: 8.56%
Cross-validation Scores: [0.56875 0.5375  0.6125  0.54375 0.5625 ]
Mean CV Score: 0.5650 (+/- 0.0528)

Classification Report:
               precision    recall  f1-score   support

    IsAbusive       0.92      0.14      0.25        77
IsProvocative       0.00      0.00      0.00        38
    IsObscene       0.00      0.00      0.00        23
 IsHatespeech       0.00      0.00      0.00        35
     IsRacist       0.00      0.00      0.00        30

    micro avg       0.92      0.05      0.10       203
    macro avg       0.18      0.03      0.05       203
 weighted avg       0.35      0.05      0.09       203
  samples avg       0.06      0.04      0.04       203

Hamming Loss: 0.1930


### 3. Building an API to consume the model

In [ ]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import nest_asyncio

# Apply nest_asyncio
nest_asyncio.apply()

# Create a FastAPI app
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, replace with specific origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define the request model
class PredictionRequest(BaseModel):
    text: str
    video_id: int

@app.post('/predict')
async def predict(request: PredictionRequest):
    try:
        text_tfidf = vectorizer.transform([request.text])
        input_data = np.hstack((text_tfidf.toarray(), [[request.video_id]]))
        prediction = model.predict(input_data)[0]
        return {'is_toxic': bool(prediction)}
    except Exception as e:
        return {'error': str(e)}

# Add a health check endpoint
@app.get('/health')
async def health():
    return {'status': 'ok'}

if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port=8000)  # Changed host to localhost

INFO:     Started server process [7204]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52271 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52271 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7204]
